In [1]:
import pandas as pd
import numpy as np
import urllib.request
from urllib import request
import bs4
from bs4 import BeautifulSoup

master_df = pd.read_csv('EPAR_table_4.csv',header = 0,skiprows = 8)
df = master_df[master_df['Category'] == 'Human']
df2 = df[df['Authorisation status'] == 'Authorised']

url_list = df2['URL'].tolist()
print(url_list)

['https://www.ema.europa.eu/en/medicines/human/EPAR/ryeqo', 'https://www.ema.europa.eu/en/medicines/human/EPAR/yesafili', 'https://www.ema.europa.eu/en/medicines/human/EPAR/leflunomide-medac', 'https://www.ema.europa.eu/en/medicines/human/EPAR/filgrastim-hexal', 'https://www.ema.europa.eu/en/medicines/human/EPAR/opzelura', 'https://www.ema.europa.eu/en/medicines/human/EPAR/ziextenzo', 'https://www.ema.europa.eu/en/medicines/human/EPAR/qutenza', 'https://www.ema.europa.eu/en/medicines/human/EPAR/levitra', 'https://www.ema.europa.eu/en/medicines/human/EPAR/rilutek', 'https://www.ema.europa.eu/en/medicines/human/EPAR/clopidogrel-krka', 'https://www.ema.europa.eu/en/medicines/human/EPAR/hycamtin', 'https://www.ema.europa.eu/en/medicines/human/EPAR/pylobactell', 'https://www.ema.europa.eu/en/medicines/human/EPAR/fulphila-0', 'https://www.ema.europa.eu/en/medicines/human/EPAR/orladeyo', 'https://www.ema.europa.eu/en/medicines/human/EPAR/semglee', 'https://www.ema.europa.eu/en/medicines/human

In [5]:
import requests
import time

product_info_url_list = []
for i in range(len(url_list)):
    success = False
    print(i)
    while not success:   
        link = url_list[i]
        response = requests.get(link)
        if response.status_code == 200:  
            web_content = response.text
            soup = BeautifulSoup(web_content, 'html.parser')
            soups = soup.find_all('a')
            href_links = [tag.get('href') for tag in soups]
            for j in range(len(href_links)):
                if '/en/documents/product-information' in str(href_links[j]):
                    product_info_url_list.append(str(href_links[j]))
            success = True
        else:
            print('error')
            time.sleep(15)

  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
error
19
20
21
22
23
24
25
26
27
28
29
30
31
32
error
33
34
35
36
error
37
38
error
39
40
41
42
error
43
44
45
46
47
48
error
49
error
50
error
51
52
53
54
55
error
56
57
58
59
error
60
error
61
62
63
error
64
65
66
67
68
error
69
70
error
71
72
73
error
74
error
75
76
77
78
79
80
81
error
82
error
83
84
error
85
86
87
88
error
89
90
91
error
92
error
93
94
error
95
error
96
error
97
98
error
99
error
100
101
102
103
error
104
error
105
106
107
error
108
109
110
error
111
112
113
114
error
115
116
117
118
119
120
121
error
122
123
124
error
125
126
127
error
128
129
error
130
error
131
132
133
134
error
135
136
137
138
139
140
error
141
142
error
143
144
error
145
146
147
148
149
150
error
151
152
153
154
155
156
error
157
158
159
160
error
161
162
163
164
165
error
166
167
168
169
170
error
171
172
173
174
175
176
177
error
178
179
180
181
182
183
error
184
185
186
187
188
189
190
191
error
192
193
error
194
error
195
196
197
198
199
200


In [13]:
import os
os.makedirs('pdf_files', exist_ok=True)

# Iterate through the list of URLs
for i, url in enumerate(product_info_url_list[1298:], start=1298):
        # Send a GET request to the URL
    success = False
    while not success:
        response = requests.get('https://www.ema.europa.eu'+ url)
    
        # Check if the request was successful
        if response.status_code == 200:
                # Define the PDF file name
            pdf_file_name = os.path.join('pdf_files', f'{str(product_info_url_list[i]).replace("/","-")}')
    
                # Write the content to a PDF file
            with open(pdf_file_name, 'wb') as pdf_file:
                    pdf_file.write(response.content)
    
            print(f"Downloaded: {pdf_file_name}")
            success = True
        else:
            print('error')
            time.sleep(15)

Downloaded: pdf_files/-en-documents-product-information-capecitabine-teva-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-oslif-breezhaler-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-hirobriz-breezhaler-epar-product-information_en.pdf
error
Downloaded: pdf_files/-en-documents-product-information-xoterna-breezhaler-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-icatibant-accord-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-onureg-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-ultibro-breezhaler-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-bortezomib-fresenius-kabi-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-product-information-bropair-spiromax-epar-product-information_en.pdf
Downloaded: pdf_files/-en-documents-produc

In [11]:
import pdfplumber
import os
keywords = ['children','pediatric','pregnant','childbearing','breastfeeding']
useful_info = []
files = os.listdir('pdf_files')

# Filter out only the PDF files
pdf_files = [f for f in files if f.endswith('.pdf')]
i=0
# Iterate through each PDF file
for pdf_file in pdf_files:
    print(i)
    useful_info_row = []
    pdf_path = os.path.join('pdf_files', pdf_file)
    useful_info_row.append(pdf_path.split('-')[5])
    try:
        with pdfplumber.open(pdf_path) as pdf:
            pdf_text = []
            for page in pdf.pages:
                pdf_text.append(page.extract_text())
            for j in range(len(pdf_text)):
                text = pdf_text[j]
                sentence_list = text.split('.')
                for k in range(len(sentence_list)):
                    if any(keyword in sentence_list[k] for keyword in keywords):
                        sentence_clean = sentence_list[k].replace('\n','').lstrip()
                        useful_info_row.append(sentence_clean)
        useful_info.append(useful_info_row)
        i=i+1
    except Exception:
        i=i+1
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
final_info_list = []
for i in range(len(useful_info)):
    lst = useful_info[i]
    new_list = [lst[0],lst[1:]]
    final_info_list.append(new_list)

print(len(final_info_list))

1347


In [19]:
import pandas as pd
final_df = pd.DataFrame(final_info_list, columns = ['drug name', 'relevant info'])
print(final_df)
final_df.to_csv('pediatric and pregnancy info for EMA drugs.csv')

         drug name                                      relevant info
0         entresto  [Paediatric heart failureEntresto is indicated...
1       filgrastim  [The safety and efficacy of filgrastim are sim...
2        ifirmasta  [Paediatric populationThe safety and efficacy ...
3           myocet  [Paediatric populationThe safety and efficacy ...
4         imatinib  [Posology for CML in children and adolescentsD...
...            ...                                                ...
1342      carbaglu  [• Patients with moderate renal impairment (GF...
1343      lumeblue  [Paediatric populationThe safety and efficacy ...
1344      rezolsta  [Therefore, therapy with REZOLSTA should not b...
1345  eptifibatide  [Paediatric populationThe safety and efficacy ...
1346        axumin  [Close contact with infants and pregnant women...

[1347 rows x 2 columns]
